In [1]:
%load_ext autoreload
%load_ext watermark

In [2]:
%autoreload 2

In [3]:
%watermark -ntz -p matplotlib,numpy,pandas,scipy

Tue Aug 09 2022 12:36:03 India Standard Time 

matplotlib 3.2.1
numpy 1.18.5
pandas 1.0.4
scipy 1.4.1


In [4]:
import datetime
import math
from pathlib import Path
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

import traffic_exercise.analysis

---
# Exercise 5: Explore Correlations
In previous exercises,
you have investigated weather and traffic data separately.
In this exercise,
you will combine the datasets
and investigate correlations between them.
At the end of this exercise,
you should understand how traffic levels
correlate to temperature and weather
on the same
and previous days.
At this point,
you should be able to form conclusions
about how well traffic levels can be predicted
by weather and traffic data
for some or all counting stations,
argue why the trends are not constant across Cumbria
and argue whether more data is required
(and of what form that data should be)
to complete further predictive analysis.

## Learning objectives
Objectives which _may_ be met during this exercise.

- I can measure simple relationships between data points (_STAT 1_)
- I can apply statistical tests to evaluate the significance of the relationsip (_STAT 1_)
- I can identify shortcomings in the data and suggest ways to improve the analysis (_STAT 1_)
- I can select the most appropriate visualisation technique for displaying complex relationships (_DATA 2_)

_Refer to the [exercise document](../references/exercise_background.md#development-objectives) for more information on objectives_


---
## Step 1: Load data

**Tasks:**
- Load traffic data
- Add columns for `month`, `weekday`, `hour`
- Turn `Date` column into date only (no time)
- Repeat with weather data

In [5]:
traffic_data_path = Path.cwd().resolve().parent /'analysis' / "data" / "interim" / "counter_data.csv"
weather_data_path = Path.cwd().resolve().parent /'analysis'/ "data" / "interim" / "weather_data_i.csv"

In [6]:
traffic_df = pd.read_csv(traffic_data_path)

traffic_df["Date" ] = pd.to_datetime(traffic_df["Date"])

#Original time is end of a timespan, subtract 1 hour to get start of period. 
#This reduces drawing artifacts where the last hour of each day has next days date.
traffic_df["Date"] = traffic_df["Date"] - pd.Timedelta(hours=1)

# TODO add month, weekday, hour columns
# TODO turn "Date" column into date only (i.e. year, month, day)
traffic_df['Month'] = traffic_df['Date'].apply(lambda x: x.month)
traffic_df['Day'] = traffic_df['Date'].apply(lambda x: x.day)
traffic_df['Weekday'] = traffic_df['Date'].apply(lambda x: x.dayofweek)
traffic_df["Hour"] = traffic_df["Date"].apply(lambda x: x.hour)
traffic_df['Date'] = traffic_df['Date'].apply(lambda d: d.date())
traffic_df.describe(include=["object", "int64", "datetime64[ns]"])

,Date,Hour Ending,Special day,Counter ID,Counts,Month,Day,Weekday,Hour
count,186624,186624.000000,45120,186624.000000,186624.000000,186624.000000,186624.000000,186624.000000,186624.000000
unique,365,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
top,2019-06-21,NaN,o,NaN,NaN,NaN,NaN,NaN,NaN
freq,576,NaN,40416,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,12.500000,NaN,44059.051569,116.501983,6.636574,15.743699,3.002829,11.500000
std,NaN,6.922205,NaN,13443.770875,181.584278,3.381432,8.760233,2.000775,6.922205
min,NaN,1.000000,NaN,20011.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,NaN,6.750000,NaN,30023.000000,8.000000,4.000000,8.000000,1.000000,5.750000
50%,NaN,12.500000,NaN,50011.000000,49.000000,7.000000,16.000000,3.000000,11.500000
75%,NaN,18.250000,NaN,50077.000000,137.000000,10.000000,23.000000,5.000000,17.250000


Load weather data

In [7]:
weather_df = pd.read_csv(weather_data_path)
weather_df["Date"] = pd.to_datetime(weather_df["Date"])
weather_df["Month"] = weather_df["Date"].apply(lambda d: d.month)
weather_df["Day"] = weather_df["Date"].apply(lambda d: d.weekday)
weather_df["Hour"] = weather_df["Date"].apply(lambda d: d.hour)
weather_df["Date"] = weather_df["Date"].apply(lambda d: d.date())

In [8]:
traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186624 entries, 0 to 186623
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Date         186624 non-null  object
 1   Hour Ending  186624 non-null  int64 
 2   Special day  45120 non-null   object
 3   Counter ID   186624 non-null  int64 
 4   Counts       186624 non-null  int64 
 5   Month        186624 non-null  int64 
 6   Day          186624 non-null  int64 
 7   Weekday      186624 non-null  int64 
 8   Hour         186624 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 12.8+ MB


In [9]:
traffic_df

,Date,Hour Ending,Special day,Counter ID,Counts,Month,Day,Weekday,Hour
0,2019-01-01,1,bo,20011,12,1,1,1,0
1,2019-01-01,2,bo,20011,10,1,1,1,1
2,2019-01-01,3,bo,20011,2,1,1,1,2
3,2019-01-01,4,bo,20011,8,1,1,1,3
4,2019-01-01,5,bo,20011,2,1,1,1,4
...,...,...,...,...,...,...,...,...,...
186619,2019-12-31,20,o,60006,6,12,31,1,19
186620,2019-12-31,21,o,60006,6,12,31,1,20
186621,2019-12-31,22,o,60006,2,12,31,1,21
186622,2019-12-31,23,o,60006,0,12,31,1,22


In [16]:
weather_df = weather_df[[x for x in weather_df.columns if x not in ['Month','Day','Hour']]]

In [17]:
weather_df

,Date,Rainfall (mm),MaxApparentTemp (degC)
0,2019-01-01,0.0,7.7
1,2019-01-02,0.0,0.4
2,2019-01-03,0.0,1.0
3,2019-01-04,0.0,2.8
4,2019-01-05,0.0,3.4
...,...,...,...
360,2019-12-27,0.6,5.8
361,2019-12-28,0.3,6.8
362,2019-12-29,0.0,6.3
363,2019-12-30,0.0,7.3


---
## Step 2: Split data

Before we continue with our analysis,
it's important to remove bias that could influence our conclusions

**Tasks:**
- Split data into train, validation and test datasets
    - Consider why you're doing this in the data investigation stage, not data or model engineering
        - What could go wrong if we used all data in this step?
        - Should we have done this step earlier? If so, what are the possible problems and what can we do about it?
    - How you split the data (e.g. by month, a proportion of each month, by counting site) will depend on which data features you wish to use and what analyses you think you can get out of the data
- Which dataset should you use for the rest of this exercise?
- Save the split datasets in `data/interim/`

In [20]:
traffic_df['Counter ID'].value_counts()

60003    8712
50010    8712
60004    8712
50009    8712
50078    8688
50053    8688
50054    8688
50077    8688
30021    8400
30023    7944
60006    7824
60005    7824
20053    7800
20054    7800
50003    7608
50043    7608
50004    7584
50044    7584
30022    7584
30024    7296
50012    6216
50011    6216
20012    5880
20011    5856
Name: Counter ID, dtype: int64

In [28]:
train_counters = [60003,50010,50078,50053,30021,30023,20054,50003,50044,30022,50011,20012,60006,20053,20011,30024]
validation_counters = [50077, 60004,50012,60005]
test_counters = [50004,50009,50043,50054]
#train = traffic_df[traffic_df['Counter ID'].isin([])]

In [29]:
[counter for counter in traffic_df['Counter ID'].unique() if (counter not in train_counters) and (counter not in validation_counters) and (counter not in test_counters)]

[]

---
## Step 3: Calculate monthly correlations
We have provided a shell of a function
which calculated Pearson correlation
between rainfall/temperature
and traffic count,
for a particular month.

**Tasks:**
- Complete function `calculate_pearson` in `src/traffic_exericse/analysis/correlation.py`
- For all counters,
plot _significant_ (p < 0.05) correlations against month,
for both rainfall and temperature
- What do you notice about the correlations over rainfall and temperature to traffic? What should we do about that?
- Are certain months more strongly correlated with a weather pattern?
- Are certain counting stations more strongly correlated?
- **\[Extension]** Is it valid to perform many correlation checks in this way? What do we have to be careful of? What would be a more robust method?

Plot correlations by month

---
## Step 4: Weekends only
It would be logical to assume that
weekend travel is more correlated with weather
because it is less driven by work commitments than weekday travel.

**Tasks:**
- Add a boolean hyperparam (default `False`) called `weekend_only`
to `calculate_pearson`,
which only includes data from Friday, Saturday, Sunday if `True`
- How do correlations change if we only include weekends?
- Are Friday, Saturday, Sunday equally dependent on weather?
- **\[Extension]** Consider what date parameters we should include in a dataset
on which to train a predictive model.
For example,
do we need to know the exact day,
or would a `weekend`/`weekday` boolean flag capture the pertinent information?

---
## Step 5: Investigate other correlations

**Tasks:**
- Extend `calculate_pearson` to accept a parameter which determines over what timescale (e.g. `month`, `weekday`, `hour`) correlations should be measured
- Investigate correlations between weather and days and hours, as you've done previously with months and weekends
- Investigate correlations between traffic levels and weather _on the previous day_
- Given the number of parameter pairs you have looked at (month, weekday, hour, weather, counting station), it is likely that you have seen some statistically significant relationships
    - Can we take these relationships as correct? How can we verify them?
    - Where did we misstep in our analysis?
        - _Hint: research "p-hacking"_

---
# Review

After this exercise:

- [ ] I can measure simple relationships between data points (_STAT 1_)
- [ ] I can apply statistical tests to evaluate the significance of the relationsip (_STAT 1_)
- [ ] I can identify shortcomings in the data and suggest ways to improve the analysis (_STAT 1_)
- [ ] I can select the most appropriate visualisation technique for displaying complex relationships (_DATA 2_)